幹部內訓 2021.01.16.
# **文字探勘 Sesssion 1** 

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Portfolio/DAC_Internal_Training')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import re
import numpy as np
import pandas as pd
import jieba
import jieba.posseg as pseg

## **基本字串處理**

### 搜尋
- pattern in string<br/>
只會回傳 True / False<br/><br/>
- re.match(pattern, string)<br/>
從第一個字元開始比對，如果沒有相符的就會直接傳回 None，比對到第一個 match 就結束<br/><br/>
- re.search(pattern, string)<br/>
只要string當中有出現與pattern相符的內容就會被比對出來，比對到第一個 match 就結束<br/><br/>
- re.findall(pattern, string)<br/>
可以比對完所有相符的 match，會回傳一個比對出來的字串 list<br/><br/>
- re.finditer(pattern, string)<br/>
可以比對完所有相符的 match，會回傳一個可迭代的物件，用 for loop 看裡面就是 MatchObject

In [3]:
sample = "咖啡是一種豆漿，茶是一種蔬菜湯"
print("origin                 :", sample)
print("")
print("pattern in string      :", "是一種" in sample) # 有找到
print("re.match()             :", re.match("是一種", sample)) # 找不到
print("re.search()            :", re.search("是一種", sample)) # 有找到
print("re.search() from start :", re.search("^是一種", sample)) # 和 match 意思一樣

origin                 : 咖啡是一種豆漿，茶是一種蔬菜湯

pattern in string      : True
re.match()             : None
re.search()            : <re.Match object; span=(2, 5), match='是一種'>
re.search() from start : None


In [4]:
print("origin :", sample)
print("")
print(re.search("是一種", sample))
if re.search("是一種", sample):
    print("start  :", re.search("是一種", sample).start())
    print("end    :", re.search("是一種", sample).end())
    print("span   :", re.search("是一種", sample).span())

origin : 咖啡是一種豆漿，茶是一種蔬菜湯

<re.Match object; span=(2, 5), match='是一種'>
start  : 2
end    : 5
span   : (2, 5)


In [5]:
print("origin        :", sample)
print("")
print("re.findall()  :", re.findall("是一種", sample))
print("re.finditer() :", re.finditer("是一種", sample))

origin        : 咖啡是一種豆漿，茶是一種蔬菜湯

re.findall()  : ['是一種', '是一種']
re.finditer() : <callable_iterator object at 0x7fa074426390>


In [6]:
print("origin :", sample)
print("")
for i in re.finditer("是一種", sample):
    print(i)
    print("start  :", i.start())
    print("end    :", i.end())
    print("span   :", i.span())

origin : 咖啡是一種豆漿，茶是一種蔬菜湯

<re.Match object; span=(2, 5), match='是一種'>
start  : 2
end    : 5
span   : (2, 5)
<re.Match object; span=(9, 12), match='是一種'>
start  : 9
end    : 12
span   : (9, 12)


### 計數
- string.count(pattern)<br/>
直接比對原本的字串中 pattern 出現多少次<br/><br/>
- len()<br/>
直接計算 re.findall() 出來的 list 長度就知道有多少配對到的字串了

In [7]:
print("origin         :", sample)
print("")
print("string.count() : find", sample.count("是一種"))
print("len()          : find", len(re.findall("是一種", sample)))

origin         : 咖啡是一種豆漿，茶是一種蔬菜湯

string.count() : find 2
len()          : find 2


### 分割
- string.split(sep)
- re.split(pattern, string)

In [8]:
print("origin         :", sample)
print("")
print("string.split() :", sample.split("，"))
print("re.split()     :", re.split("，|。", sample))

origin         : 咖啡是一種豆漿，茶是一種蔬菜湯

string.split() : ['咖啡是一種豆漿', '茶是一種蔬菜湯']
re.split()     : ['咖啡是一種豆漿', '茶是一種蔬菜湯']


### 取代
- re.sub(pattern, repl, string, count=0)

In [9]:
print("origin          :", sample)
print("")
print("re.sub()        :", re.sub("是一種", "才不是", sample))
print("re.sub(count=1) :", re.sub("是一種", "才不是", sample, count=1)) # 只取代一次

origin          : 咖啡是一種豆漿，茶是一種蔬菜湯

re.sub()        : 咖啡才不是豆漿，茶才不是蔬菜湯
re.sub(count=1) : 咖啡才不是豆漿，茶是一種蔬菜湯


### 正規表示法 (Regular Expression)
- 好用的Regex語法網站: Regular Expression 101: https://regex101.com/

- 取代掉指定的字元

In [10]:
sample_e = "(1) Coffee is a kind of soy milk.   (2) Tea is a kind of vegetable soup."
print("origin                   :", sample_e)
print("")

# 直接配對完全符合的字元 (和前面一樣)
print("replace 'kind'           :", re.sub(r"kind",      "____", sample_e))
print("")

# 利用"[]"表示要配對一個集合中的所有字元，也可以利用"-"表示一個範圍
print("replace 'aeiou'          :", re.sub(r"[aeiou]",   "_", sample_e))
print("replace 'abcde'          :", re.sub(r"[a-e]",     "_", sample_e))
print("")

# 利用 \d \w \s 取代掉數字、英數字及空白，\D \W \S 分別表示取代數字、英數字及空白以外的字元
print("replace digit            :", re.sub(r"\d",        "_", sample_e))
print("replace digit            :", re.sub(r"[0-9]",     "_", sample_e)) # [0-9] 與 \d 意義相同
print("")

print("replace digit & alphabet :", re.sub(r"\w",        "_", sample_e))
print("replace digit & alphabet :", re.sub(r"[A-z0-9]",  "_", sample_e)) # [A-z0-9] 與 \w 意義相同
print("replace not \w character :", re.sub(r"\W",        "_", sample_e)) # 大寫代表「不屬於」這類的字元
print("")

print("replace space            :", re.sub(r"\s",        "_", sample_e))
print("")

# 利用"."表示任意字元
print("replace anything         :", re.sub(r".",        "_", sample_e))
print("")

# 利用"\"跳脫字元意義
print("replace dot              :", re.sub(r"\.",        "_", sample_e))

origin                   : (1) Coffee is a kind of soy milk.   (2) Tea is a kind of vegetable soup.

replace 'kind'           : (1) Coffee is a ____ of soy milk.   (2) Tea is a ____ of vegetable soup.

replace 'aeiou'          : (1) C_ff__ _s _ k_nd _f s_y m_lk.   (2) T__ _s _ k_nd _f v_g_t_bl_ s__p.
replace 'abcde'          : (1) Coff__ is _ kin_ of soy milk.   (2) T__ is _ kin_ of v_g_t__l_ soup.

replace digit            : (_) Coffee is a kind of soy milk.   (_) Tea is a kind of vegetable soup.
replace digit            : (_) Coffee is a kind of soy milk.   (_) Tea is a kind of vegetable soup.

replace digit & alphabet : (_) ______ __ _ ____ __ ___ ____.   (_) ___ __ _ ____ __ _________ ____.
replace digit & alphabet : (_) ______ __ _ ____ __ ___ ____.   (_) ___ __ _ ____ __ _________ ____.
replace not \w character : _1__Coffee_is_a_kind_of_soy_milk_____2__Tea_is_a_kind_of_vegetable_soup_

replace space            : (1)_Coffee_is_a_kind_of_soy_milk.___(2)_Tea_is_a_kind_of_vegetable_s

- 取代指定的次數

In [11]:
sample_e1 = "(1) Coffee is a kind of sooooy milk."
print("origin              :", sample_e1)
print("")

print("find 'eo'           :", re.sub(r"[eo]",      "/", sample_e1))
print("find 0 or 1 'eo'    :", re.sub(r"[eo]?",     "/", sample_e1))
print("find 1 or more 'eo' :", re.sub(r"[eo]+",     "/", sample_e1))
print("find 0 or many 'eo' :", re.sub(r"[eo]*",     "/", sample_e1))
print("find 1 to 3 'eo'    :", re.sub(r"[eo]{1,3}", "/", sample_e1))

origin              : (1) Coffee is a kind of sooooy milk.

find 'eo'           : (1) C/ff// is a kind /f s////y milk.
find 0 or 1 'eo'    : /(/1/)/ /C//f/f/// /i/s/ /a/ /k/i/n/d/ //f/ /s/////y/ /m/i/l/k/./
find 1 or more 'eo' : (1) C/ff/ is a kind /f s/y milk.
find 0 or many 'eo' : /(/1/)/ /C//f/f// /i/s/ /a/ /k/i/n/d/ //f/ /s//y/ /m/i/l/k/./
find 1 to 3 'eo'    : (1) C/ff/ is a kind /f s//y milk.


- 取代幾組特定的組合

In [12]:
print("origin                     :", sample_e)
print("")

print("replace a set of characters :", re.sub(r"(Coffee|Tea)",        "___", sample_e))

origin                     : (1) Coffee is a kind of soy milk.   (2) Tea is a kind of vegetable soup.

replace a set of characters : (1) ___ is a kind of soy milk.   (2) ___ is a kind of vegetable soup.


- 以附近出現的字搜尋 (look around)：(?=)、(?!)、(?<=)、(?<!)

In [13]:
print("origin  :", sample_e)
print("")


print("克漏字 1 :", re.sub(r"(?<!\s\w)\w+",              "__", sample_e))
print("克漏字 2 :", re.sub(r"(?<=\s\w)\w+(?=\w\s)",      "__", sample_e))
print("克漏字 3 :", re.sub(r"(?<=\s\w)\w+(?=\w(\s|\.))", "__", sample_e))

origin  : (1) Coffee is a kind of soy milk.   (2) Tea is a kind of vegetable soup.

克漏字 1 : (__) __ __ __ __ __ __ __.   (__) __ __ __ __ __ __ __.
克漏字 2 : (1) C__e is a k__d of s__y milk.   (2) T__a is a k__d of v__e soup.
克漏字 3 : (1) C__e is a k__d of s__y m__k.   (2) T__a is a k__d of v__e s__p.


- 只搜尋頭尾 (anchor): ^、$

In [14]:
print("origin                                    :", sample_e)
print("")

print("find not space characters at the begining :", re.sub(r"^\S+", "_", sample_e))
print("find not space characters at the end      :", re.sub(r"\S+$", "_", sample_e))

origin                                    : (1) Coffee is a kind of soy milk.   (2) Tea is a kind of vegetable soup.

find not space characters at the begining : _ Coffee is a kind of soy milk.   (2) Tea is a kind of vegetable soup.
find not space characters at the end      : (1) Coffee is a kind of soy milk.   (2) Tea is a kind of vegetable _


- 練習題：有效email地址判定

In [15]:
email_list = ["ntu.data.analytics@gmail.com", 
              "ntu_data_analytics@gmail.com", 
              "ntu+data+analytics@gmail.com", 
              "ntu.dåtå.ånålytiçs@gmail.com", 
              "ntu.data.analytics@gmail.", 
              "ntu.data.analytics@com"]
email_pattern = "^\w+((-\w+)|(\.\w+))*\@[A-Za-z0-9]+((\.|-)[A-Za-z0-9]+)*\.[A-Za-z]+$"
for email in email_list:
    if re.search(email_pattern, email):
        print(re.search(email_pattern, email).group(0))
    else: 
        print("---- WARNING: {} is not a valid email address! ----".format(email))

ntu.data.analytics@gmail.com
ntu_data_analytics@gmail.com
---- WARNING: ntu+data+analytics@gmail.com is not a valid email address! ----
ntu.dåtå.ånålytiçs@gmail.com
---- WARNING: ntu.data.analytics@gmail. is not a valid email address! ----
---- WARNING: ntu.data.analytics@com is not a valid email address! ----


## **結巴**

### 斷詞 + 加入詞性

In [16]:
# 載入辭典
jieba.set_dictionary('./jupyternb/dict.txt.big')

In [17]:
sample_cut = "咖啡是一種豆漿，茶是一種蔬菜湯，科學麵是一種沙西米"
print("origin                 :", sample_cut)
print("")
print("cut sentence           :", jieba.lcut(sample_cut))
print("cut sentence with flag :", [(word, flag) for (word, flag) in pseg.cut(sample_cut)])

Building prefix dict from /content/drive/MyDrive/Portfolio/DAC_Internal_Training/jupyternb/dict.txt.big ...
Loading model from cache /tmp/jieba.u3d3efe839fba6c46342422c437a663f0.cache


origin                 : 咖啡是一種豆漿，茶是一種蔬菜湯，科學麵是一種沙西米



Loading model cost 0.537 seconds.
Prefix dict has been built successfully.


cut sentence           : ['咖啡', '是', '一種', '豆漿', '，', '茶', '是', '一種', '蔬菜湯', '，', '科學', '麵', '是', '一種', '沙西米']
cut sentence with flag : [('咖啡', 'n'), ('是', 'v'), ('一種', 'x'), ('豆漿', 'n'), ('，', 'x'), ('茶', 'n'), ('是', 'v'), ('一種', 'x'), ('蔬菜', 'n'), ('湯', 'n'), ('，', 'x'), ('科', 'n'), ('學', 'v'), ('麵', 'zg'), ('是', 'v'), ('一種', 'x'), ('沙西米', 'ns')]


### 加入自定義辭典

In [18]:
# jieba.load_userdict("userdict.txt")  ## 自己編寫一個 txt 檔，用換行把每個詞分開
jieba.add_word("科學麵")

print("origin       :", sample_cut)
print("")
print("cut sentence :", jieba.lcut(sample_cut))

origin       : 咖啡是一種豆漿，茶是一種蔬菜湯，科學麵是一種沙西米

cut sentence : ['咖啡', '是', '一種', '豆漿', '，', '茶', '是', '一種', '蔬菜湯', '，', '科學麵', '是', '一種', '沙西米']


### 移除停用字

In [19]:
# stop_words = open('stop_words.txt', 'r').readlines()  ## 一樣可以自己匯入檔案
stop_words = ["，"]
cut_words = []
for w in jieba.lcut(sample_cut):
    if w not in stop_words:
        cut_words.append(w)
        
print("origin                   :", sample_cut)
print("")        
print("cut and remove stopwords :", cut_words)

origin                   : 咖啡是一種豆漿，茶是一種蔬菜湯，科學麵是一種沙西米

cut and remove stopwords : ['咖啡', '是', '一種', '豆漿', '茶', '是', '一種', '蔬菜湯', '科學麵', '是', '一種', '沙西米']


# **PTT文本分析**

### PTT資料斷詞

In [22]:
BOARD_LIST = ["BoyGirl", "HatePolitics", "NBA", "Stock"]

BoyGirl = pd.read_csv("./ptt_csv/Boy_Girl_200.csv").drop(columns = "Unnamed: 0")
BoyGirl["source"] = "BoyGirl"
HatePolitics = pd.read_csv("./ptt_csv/HatePolitics_200.csv").drop(columns = "Unnamed: 0")
HatePolitics["source"] = "HatePolitics"
NBA = pd.read_csv("./ptt_csv/NBA_200.csv").drop(columns = "Unnamed: 0")
NBA["source"] = "NBA"
Stock = pd.read_csv("./ptt_csv/Stock_200.csv").drop(columns = "Unnamed: 0")
Stock["source"] = "Stock"
# ptt = Gossiping.append(BoyGirl)
# ptt = ptt.append(joke).reset_index().drop(columns = "index")

# ptt = ptt.dropna(how = "any")

ptt = BoyGirl[0:1000].append(HatePolitics[0:1000])
ptt = ptt.append(NBA[0:1000])
ptt = ptt.append(Stock[0:1000]).reset_index().drop(columns = "index").rename(columns={"標題":"topic", "時間": "time", "內容": "content"})

ptt = ptt.dropna(how = "any")
ptt = ptt[~ptt['topic'].str.contains('公告')]
ptt['content'] = ptt['content'].str.replace('原文標題|完整新聞標題|原文連結|新聞網址|詳細報導請見|新聞來源|編輯|報導|發布時間|發佈時間|最後更新時間|原文內容|完整新聞內容|心得/評論|附註、心得、想法', '')

In [23]:
print(ptt.shape)
ptt.sample(5)

(3939, 4)


,topic,time,content,source
2878,[花邊] 快艇考慮過Lowry後還是決定不交易,2021-03-16 00:47:28,"<a href=""https://reurl.cc/3NM988"" rel=""nofollo...",NBA
2109,[新聞] 昔日雷霆大鎖宣布退役 聯盟「06梯」只,2021-04-02 07:14:22,昔日雷霆大鎖宣布退役 聯盟「06梯」只剩7人\n\n記者黃建霖/綜合\n\n千禧年第一個十年...,NBA
531,[心情] 白目男友說大姨媽是藍色的.....,2021-03-15 17:16:59,"剛看訪問大學生的影片\n<a href=""https://tinyurl.com/4jwpr...",BoyGirl
3068,Re: [其他] 0050每年最高與最低點進場的報酬差異,2021-04-03 02:38:23,"\n""因為現在最高，所以過去任何一個點都比現在低。""\n\n能把X話說得那麼認真，真是不簡單...",Stock
3660,[新聞] 漲潮時間已過 長榮長賜輪脫困希望落空,2021-03-28 12:02:20,"\n： 漲潮時間已過 長榮長賜輪脫困希望落空\n： <a href=""https://udn...",Stock


In [24]:
jieba.set_dictionary('./jupyternb/dict.txt.big')
stop_words = [re.sub('\n', '', stop) for stop in open('./jupyternb/stop_words.txt', 'r').readlines()]
symbols = r"，|。|！|？|：|⋯⋯|{0}|\(|\)|（|）|=|\+|「|」|『|』|《|》|〈|〉|【|】|〖|〗|［|］|<|>|/|\.|\n|[A-z0-9]".format('""')
flags = ["n", "nr", "nt", "ng", "nrt", "v", "vg", "d", "a"]

for r in ptt.iterrows():
    print("{} / {}".format(r[0]+1, ptt.shape[0]))
    content = r[1]['content']
    content = re.sub(symbols, " ", content)
    content_cutlist = []
    for w, f in [(word, flag) for (word, flag) in pseg.cut(content)]:
        if w not in stop_words and f in flags and len(w) >= 2:
            # print(w, f)
            content_cutlist.append(w)
    content_cut = " ".join(content_cutlist)
    # print(content, "\n===================")
    ptt.loc[r[0], "content_cut"] = content_cut

Exception: ignored

In [ ]:
ptt

## **關鍵字選取**

- 將四個版的文章斷詞結果 join 成一個字串，以利用詞頻與 tfidf 值選取四個版的關鍵字

In [ ]:
corpus = [
    " ".join(ptt[ptt["source"] == BOARD_LIST[0]]["content_cut"]), 
    " ".join(ptt[ptt["source"] == BOARD_LIST[1]]["content_cut"]), 
    " ".join(ptt[ptt["source"] == BOARD_LIST[2]]["content_cut"]), 
    " ".join(ptt[ptt["source"] == BOARD_LIST[3]]["content_cut"]) 
]
corpus

### 詞頻

In [ ]:
# 忽略在 80% 文本中都有出現的詞彙 (即去除高頻率詞彙)
# 字詞至少出現在 5 篇文章中才進行轉換
vectorizer = CountVectorizer(max_df = 0.8, min_df = 2)
count = vectorizer.fit_transform(corpus)
# print(vectorizer.get_feature_names())  
print(vectorizer.vocabulary_) # key為字詞，value為字詞id
print("=="*30)
print(count.toarray())

In [ ]:
term_freq = pd.DataFrame(count.toarray(), 
                         columns = vectorizer.get_feature_names(), 
                         index = BOARD_LIST)
term_freq

- 詞頻長條圖

In [ ]:
for board in BOARD_LIST:
    plt.clf()
    top_15 = term_freq.T[board].sort_values(ascending=False).head(15)
    # print(top_15)
    freq_plot = top_15.plot(kind='barh')
    freq_plot.set_yticklabels(top_15.index, fontproperties=font)
    freq_plot.invert_yaxis() # 沒有加上.invert_yaxis()的話最大值會在下
    plt.title(board)
    plt.show()

### TF-IDF

- 可以利用詞頻的結果轉換

In [ ]:
transformer = TfidfTransformer()
tfidf_matrix = transformer.fit_transform(count)
print(tfidf_matrix.toarray())

In [ ]:
tfidf = pd.DataFrame(tfidf_matrix.toarray(), 
                     columns = vectorizer.get_feature_names(), 
                     index = BOARD_LIST)
tfidf

- 也可以直接跑出一個 TFIDF matrix

In [ ]:
# max_df=0.5 如果詞彙出現在50%以上的文本就不考慮
# min_df=5 如果詞彙出現次數少於5次就不考慮, min_df=0.1 如果詞彙在10%以下的文件中出現就不考慮
vectorizer2 = TfidfVectorizer() 
vectorizer2.fit(corpus)

## 得到tfidf vector
tfidf2 = vectorizer2.transform(corpus)
print(vectorizer2.vocabulary_)
print("=="*30)
print(tfidf2.toarray())

In [ ]:
pd.DataFrame(tfidf2.toarray(), 
             columns = vectorizer2.get_feature_names(), 
             index = BOARD_LIST)

### 文字雲

In [ ]:
for board in BOARD_LIST:
    freq_dic = dict(
        zip(tfidf.columns, 
            tfidf.loc[board])
    )

    cloud = WordCloud(font_path='./jf-openhuninn-1.0.ttf').generate_from_frequencies(freq_dic)
    plt.figure()
    plt.imshow(cloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(board)
    plt.show()

## **詞彙向量化：計數方法**

### 共現矩陣

In [ ]:
from gensim.corpora.dictionary import Dictionary
corpus_dictionary = Dictionary([stc.split() for stc in list(corpus)])
token2id = Dictionary([stc.split() for stc in list(corpus)]).token2id
id2token = dict(corpus_dictionary)
id_corpus = [list(map(lambda x: token2id[x], sentence.split())) for sentence in corpus]

In [ ]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    # initialize co-occurrence matrix
    co_matrix = np.zeros(shape=(vocab_size, vocab_size), dtype=np.int32)
    
    for sentence in corpus:
        sentence_size = len(sentence)
    
        for idx, word_id in enumerate(sentence):
            for i in range(1, window_size+1):
                left_idx = idx - i
                right_idx = idx + i

                if left_idx >= 0:
                    left_word_id = sentence[left_idx]
                    co_matrix[word_id, left_word_id] += 1

                if right_idx < sentence_size:
                    right_word_id = sentence[right_idx]
                    co_matrix[word_id, right_word_id] += 1
    np.fill_diagonal(co_matrix, 0)
    
    return co_matrix

co_matrix = create_co_matrix(id_corpus, len(token2id), 5)
co_matrix

In [ ]:
pd.DataFrame(co_matrix, 
            columns = id2token.values(), 
            index = id2token.values())
# idx2word.values()

In [ ]:
# 選出字頻前幾高的呈現就好
high_f_word = term_freq.sum().sort_values(ascending = False)[0:30]
high_f_word

In [ ]:
coocc_df = pd.DataFrame(co_matrix, 
                           columns = id2token.values(), 
                           index = id2token.values())
coocc_top30 = coocc_df.loc[high_f_word.index, high_f_word.index]
coocc_top30

### heat map

In [ ]:
plt.rcParams['figure.figsize'] = (16, 12)
heatmap = sns.heatmap(coocc_top30, cmap='YlOrBr')
heatmap.set_xticklabels(coocc_top30.index, fontproperties=font, fontsize=12)
heatmap.set_yticklabels(coocc_top30.index, fontproperties=font, fontsize=12)

### network

In [ ]:
term_list = list(coocc_top30.index)
keys = []
values = []
for c in coocc_top30.index:
    # print("-"*3, count, "-"*3)
    for i in term_list:
        if c == i:
            pass
        else:
            keys.append((c, i))
            values.append(coocc_top30.loc[i, c])
            # print(keys[-1], values[-1])
    del term_list[0]
    
weight = dict(zip(keys, values))
weight

In [ ]:
# 製作一個 network plot 物件
G = nx.Graph()

# 由 weight 中的 value 定義網路圖連線的粗細
for k, v in weight.items():
    G.add_edge(k[0], k[1], weight=(v * 10)) # edge
all_weights = []
for (node1,node2,data) in G.edges(data=True):
        all_weights.append(data['weight']) # we'll use this when determining edge thickness

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))

# 畫 nodes
pos = nx.spring_layout(G, k=2)
nx.draw_networkx_nodes(G, pos,
                 # font_size=20,
                 node_color='black',
                 # with_labels = False,
                 ax=ax)
# 畫 edges
for weight in all_weights:
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in G.edges(data=True) if edge_attr['weight']==weight]
        width = (weight^2)*0.3*len(list(G.nodes))/max(all_weights)
        nx.draw_networkx_edges(G,pos,
                               alpha=weight*0.3*len(list(G.nodes))/(sum(all_weights)),
                               edge_color='orange',
                               edgelist=weighted_edges,
                               width=width)

# 加上文字 label
for key, value in pos.items():
    x, y = value[0]+.035, value[1]+.045
    ax.text(x, y,
            s=key,
            bbox=dict(facecolor='gray', alpha=0.45),
            horizontalalignment='center', fontsize=13,
            fontproperties=font)
    
plt.show()

### Cosine Similarity

In [ ]:
# 定義餘弦相似度
def cos_similarity(x, y, eps=1e-8):
    nx = x / (np.sqrt(np.sum(x**2)) + eps)
    ny = y / (np.sqrt(np.sum(y**2)) + eps)
    
    return np.dot(nx,ny)


cos_similarity(co_matrix[token2id['騎士']], co_matrix[token2id['火箭']])

In [ ]:
def top_k_similarity(query: str, token2id: dict, id2token: dict, word_matrix: np.ndarray, top_k: int=3):
    # handle the situation of query word not in corpus
    if query not in token2id:
        raise ValueError(f"文本中並沒有出現 {query} 這個詞喔")
    
    # handle the situation of top_k is the same as the amount of words
    if top_k >= len(token2id):
        raise ValueError(f"top_k needs to be less than the amount of words")
        
    print(f"[query] : {query}")
    query_id = token2id[query]
    query_vec = word_matrix[query_id]
    
    # calculate cosine similarity
    similarity_scores = np.zeros(len(token2id))
    for i in range(len(token2id)):
        similarity_scores[i] = cos_similarity(query_vec, word_matrix[i])

    # remove query word
    similarity_scores[query_id] = 0
    filter_token2id = dict([(k, v) for k, v in token2id.items() if k != query])
    filter_id2token = dict([(k, v) for k, v in id2token.items() if k != query_id])
    
    # sorting by similarity score
    top_k_idx = (-similarity_scores).argsort()[:top_k]
    top_k_word = [filter_id2token[word_idx] for word_idx in top_k_idx]
    
    return dict(zip(top_k_word, similarity_scores[top_k_idx]))

In [ ]:
top_k_similarity('疫情', token2id, id2token, co_matrix, 5)

## **詞彙向量化：推論方法**

### word2vec
- 參數意義<br/>
*seed*： 亂數種子<br/>
*sg*： Word2Vec 有兩種算法，CBOW 以及 Skip-gram，這裡選擇了訓練比較快的 CBOW<br/>
*window_size*： 周圍詞彙要看多少範圍<br/>
*size*： 轉成向量的維度<br/>
*min_count*： 詞頻少於 min_count 之詞彙不會參與訓練<br/>
*workers*： 訓練的並行數量<br/>
*iter*： 訓練的迭代次數<br/>
*batch_words*：每次給予多少詞彙量訓練

In [ ]:
from gensim.models import word2vec

# Settings
seed = 66
sg = 0
window_size = 10
vector_size = 100
min_count = 3
workers = 8
epochs = 5
batch_words = 10000

w2v = word2vec.Word2Vec(
    [c.split(" ") for c in ptt["content_cut"]],
    min_count = 3,
    size = 100,
    workers = 8,
    iter = 5,
    window = 10,
    sg = 0,
    seed = 66,
    batch_words = 10000
)

# model.save('word2vec.model')

### cosine similarity

In [ ]:
# print word list
for word in w2v.wv.vocab: print(word)

In [ ]:
print(w2v['疫情'].shape)

for item in w2v.most_similar('疫情'):
    print(item)

## **文本分類**

### LDA

In [ ]:
#使用LDA演算法
LDA = LatentDirichletAllocation(n_components = 4, random_state = 66)
LDA.fit(count)
for i,topic in enumerate(LDA.components_):
    print("TOP 10 WORDS For TOPIC #{}".format(i+1))
    print([vectorizer.get_feature_names()[index] for index in topic.argsort()[-10:]])

In [ ]:
corpus_for_lda = [corpus_dictionary.doc2bow(w) for w in [stc.split() for stc in list(corpus)]]
print(corpus_dictionary)
print(corpus_for_lda)

In [ ]:
LDA2 = gensim.models.LdaMulticore(corpus = corpus_for_lda,
                                  id2word = corpus_dictionary,
                                  num_topics = 4, 
                                  random_state = 66,
                                  chunksize = 30,
                                  passes = 100, 
                                  per_word_topics=True)

In [ ]:
for idx, ftopic in LDA2.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, ftopic))